In [3]:
import pandas as pd
inpatient = pd.read_csv('../project/Train_Inpatientdata.csv')

In [4]:
# Import modules for feature engineering and modelling
from sklearn.model_selection import train_test_split  # not used here, we use <2015 and >=2015 to split data
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
# from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
# from sklearn.pipeline import FeatureUnion  # not sure which one to choose, the featureunion or the columntransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression 

from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge
# for integer encoding using sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

# Set seed for reproducibility
seed1, seed2, seed3=101,67, 42

In [4]:
claimYear = pd.DatetimeIndex(inpatient['ClaimEndDt']).year

In [5]:
claimYear.unique()

Int64Index([2009], dtype='int64', name='ClaimEndDt')

In [96]:
inpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  40474 non-null  object 
 1   ClaimID                 40474 non-null  object 
 2   ClaimStartDt            40474 non-null  object 
 3   ClaimEndDt              40474 non-null  object 
 4   Provider                40474 non-null  object 
 5   InscClaimAmtReimbursed  40474 non-null  int64  
 6   AttendingPhysician      40362 non-null  object 
 7   OperatingPhysician      23830 non-null  object 
 8   OtherPhysician          4690 non-null   object 
 9   AdmissionDt             40474 non-null  object 
 10  ClmAdmitDiagnosisCode   40474 non-null  object 
 11  DeductibleAmtPaid       39575 non-null  float64
 12  DischargeDt             40474 non-null  object 
 13  DiagnosisGroupCode      40474 non-null  object 
 14  ClmDiagnosisCode_1      40474 non-null

In [5]:
outpatient = pd.read_csv('../project/Train_Outpatientdata.csv')

In [8]:
outpatient.head(25)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947
5,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
6,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
7,BENE11004,CLM252512,2009-03-18,2009-03-18,PRV56011,200,PHY346833,NaN,PHY346833,72290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
8,BENE11004,CLM322683,2009-04-25,2009-05-15,PRV56011,60,PHY372925,NaN,PHY311407,71856,...,71596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
9,BENE11004,CLM339500,2009-05-04,2009-05-16,PRV56011,500,PHY412904,NaN,PHY396473,7237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [9]:
outpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  517737 non-null  object 
 1   ClaimID                 517737 non-null  object 
 2   ClaimStartDt            517737 non-null  object 
 3   ClaimEndDt              517737 non-null  object 
 4   Provider                517737 non-null  object 
 5   InscClaimAmtReimbursed  517737 non-null  int64  
 6   AttendingPhysician      516341 non-null  object 
 7   OperatingPhysician      90617 non-null   object 
 8   OtherPhysician          195046 non-null  object 
 9   ClmDiagnosisCode_1      507284 non-null  object 
 10  ClmDiagnosisCode_2      322357 non-null  object 
 11  ClmDiagnosisCode_3      203257 non-null  object 
 12  ClmDiagnosisCode_4      125596 non-null  object 
 13  ClmDiagnosisCode_5      74344 non-null   object 
 14  ClmDiagnosisCode_6  

In [6]:
beneficiary = pd.read_csv('../project/Train_Beneficiarydata.csv')
beneficiary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

In [7]:
provider = pd.read_csv('../project/Train.csv')
provider.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provider        5410 non-null   object
 1   PotentialFraud  5410 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB


In [22]:
for id in inpatient.BeneID:
    if id in outpatient.BeneID:
        print("find duplication", id)
print('done')

done


In [19]:
len(outpatient.ClaimID.drop_duplicates())

517737

In [24]:
len(inpatient.DiagnosisGroupCode.unique())

736

In [26]:
len(inpatient.ClmDiagnosisCode_1.unique())

2254

In [27]:
len(inpatient.ClmDiagnosisCode_2.unique())

2440

In [28]:
len(inpatient.ClmDiagnosisCode_3.unique())

2428

In [29]:
len(inpatient.ClmDiagnosisCode_4.unique())

2442

In [30]:
len(inpatient.ClmDiagnosisCode_5.unique())

2375

In [31]:
len(inpatient.ClmDiagnosisCode_6.unique())

2359

In [32]:
len(inpatient.ClmDiagnosisCode_7.unique())

2311

In [33]:
len(inpatient.ClmDiagnosisCode_8.unique())

2244

In [34]:
len(inpatient.ClmDiagnosisCode_9.unique())

2095

In [35]:
len(inpatient.ClmDiagnosisCode_10.unique())

953

In [36]:
len(inpatient.ClmProcedureCode_1.unique())

1118

In [37]:
len(inpatient.ClmProcedureCode_2.unique())

298

In [38]:
len(inpatient.ClmProcedureCode_3.unique())

155

In [39]:
len(inpatient.ClmProcedureCode_4.unique())

49

In [40]:
len(inpatient.ClmProcedureCode_5.unique())

7

In [41]:
len(inpatient.ClmProcedureCode_6.unique())

1

In [42]:
len(outpatient.ClmProcedureCode_1.unique())

81

In [43]:
len(outpatient.ClmProcedureCode_2.unique())

23

In [44]:
len(outpatient.ClmProcedureCode_3.unique())

5

In [45]:
len(outpatient.ClmProcedureCode_4.unique())

3

In [46]:
len(outpatient.ClmProcedureCode_5.unique())

1

In [47]:
len(outpatient.ClmProcedureCode_6.unique())

1

In [48]:
len(outpatient.ClmDiagnosisCode_1.unique())

10355

In [49]:
len(outpatient.ClmDiagnosisCode_2.unique())

5057

In [50]:
len(outpatient.ClmDiagnosisCode_3.unique())

4449

In [51]:
len(outpatient.ClmDiagnosisCode_4.unique())

3926

In [52]:
len(outpatient.ClmDiagnosisCode_5.unique())

3413

In [53]:
len(outpatient.ClmDiagnosisCode_6.unique())

2969

In [54]:
len(outpatient.ClmDiagnosisCode_7.unique())

2636

In [55]:
len(outpatient.ClmDiagnosisCode_8.unique())

2261

In [56]:
len(outpatient.ClmDiagnosisCode_9.unique())

1895

In [57]:
len(outpatient.ClmDiagnosisCode_10.unique())

496

In [62]:
le=LabelEncoder()

class Label_Transformer(object):
    def __init__(self):
        self.label={}
    
    def fit(self, X, y=None):
        self.cat_colnames=X.columns
        df=X.copy()
        for column_name in self.cat_colnames:
            keys=(X[column_name]).unique()
            values=np.arange(len(keys))
            res = {keys[i]: values[i] for i in range(len(keys))}
            self.label[column_name]=res
            df=df.replace({column_name: self.label[column_name]})
        self.mean=df.mean()
    
    def transform(self, X, y=None):
        df = X.copy()
        for column_name in self.cat_colnames:
            df=df.replace({column_name: self.label[column_name]})
            df[column_name]=pd.to_numeric(df[column_name], errors='coerce')
        return df.fillna(self.mean)
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)   

In [8]:
inpatient['In_Out']=1
outpatient['In_Out']=0


In [9]:
# union/concat the inpatient and outpatient data
concat_in_out=pd.concat([inpatient, outpatient],axis=0)
concat_in_out.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,In_Out
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,1
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,1
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,1


In [10]:
merge_bene_df=concat_in_out.merge(beneficiary, on='BeneID', how='left')
merge_bene_df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,1,2,1,1,36000,3204,60,70
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,1,2,1,1,36000,3204,60,70
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,1,2,2,1,1,5000,1068,250,320
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,1,2,1,2,2,2,21260,2136,120,100


In [101]:
merge_bene_df.shape

(558211, 55)

In [11]:
merge_provider_df = merge_bene_df.merge(provider, on = 'Provider', how ='left')
merge_provider_df.shape

(558211, 56)

In [12]:
merge_df_transform = merge_provider_df.copy(deep=True)
display(merge_df_transform.head())


,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,2,1,1,36000,3204,60,70,Yes
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,2,1,1,36000,3204,60,70,No
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,2,1,1,36000,3204,60,70,No
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,2,2,1,1,5000,1068,250,320,No
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,2,1,2,2,2,21260,2136,120,100,No


In [14]:
# calculte claimPeriod and HospitalizedDays
merge_df_transform['ClaimPeriod'] = (pd.to_datetime(merge_df_transform.ClaimEndDt) - pd.to_datetime(merge_df_transform.ClaimStartDt)).dt.days
merge_df_transform['HospitalizedDays'] = (pd.to_datetime(merge_df_transform.DischargeDt) - pd.to_datetime(merge_df_transform.AdmissionDt)).dt.days


In [30]:
#merge_df_transform.DOD.unique()

In [17]:
#calculate age and determine if a person is deceased. If the person is still alive, 
#use '2009-12-01' as the time we calculate age

from datetime import date, datetime
def calulateAge(dob, dod, calulationDate):
    born = datetime.strptime(dob, "%Y-%m-%d").date()
    if not pd.isna(dod):
        calulationDate = datetime.strptime(dod, "%Y-%m-%d").date()
    else:
        calulationDate = datetime.strptime(calulationDate, "%Y-%m-%d").date()
    return calulationDate.year - born.year - ((calulationDate.month, calulationDate.day) < (born.month, born.day))    


merge_df_transform['Deceased'] = merge_df_transform['DOD'].apply(lambda x : 0 if pd.isna(x) else 1) 
merge_df_transform['Age'] = merge_df_transform.apply(lambda x: calulateAge(dob = x['DOB'], dod = x['DOD'], calulationDate = '2009-12-01'), axis = 1)

In [19]:
#Drop unecessary columns
dropColumnList = ['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimStartDt', 'AdmissionDt', 'DischargeDt', 'DOB', 'DOD']
merge_df_transform.drop(axis = 1, columns=dropColumnList, inplace=True)
merge_df_transform

,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,...,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,ClaimPeriod,HospitalizedDays,Age,Deceased
0,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,7866,1068.0,201,1970,...,1,36000,3204,60,70,Yes,6,6.0,66,0
1,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,6186,1068.0,750,6186,...,1,36000,3204,60,70,No,2,2.0,66,0
2,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,29590,1068.0,883,29623,...,1,36000,3204,60,70,No,3,3.0,66,0
3,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,431,1068.0,067,43491,...,1,5000,1068,250,320,No,8,8.0,95,0
4,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,78321,1068.0,975,042,...,2,21260,2136,120,100,No,17,17.0,71,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,2009-08-06,PRV53699,800,PHY364188,PHY364188,PHY385752,NaN,0.0,NaN,2163,...,2,0,0,5470,1870,No,0,NaN,57,0
558207,2009-08-29,PRV53702,400,PHY423019,PHY332284,NaN,NaN,0.0,NaN,07041,...,2,0,0,5470,1870,No,0,NaN,57,0
558208,2009-09-24,PRV53676,60,PHY361063,NaN,NaN,NaN,0.0,NaN,V570,...,2,0,0,5470,1870,No,0,NaN,57,0
558209,2009-10-18,PRV53689,70,PHY403198,NaN,PHY419379,NaN,0.0,NaN,NaN,...,2,0,0,5470,1870,No,0,NaN,57,0


In [20]:
'''Export to csv'''
# from pathlib import Path
# filepath = Path('../project/sampleData.csv')
# merge_df_transform.to_csv(filepath, index=False)